Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Notebook to Introduce Integration of SQL and Python Programming

## Credentials Preliminaries

In [ ]:
import pandas as pd
import sqlalchemy as sa
import json

The following two cells are the same as when we used cell magics for Database SQL interaction.  The result of the two cells is to compose a connection string in the variable `cstring`. This string is used to form a connection to the database.

In [ ]:
def getCreds(filename, subset, defaults = {}):
    """ Use `filename` to look for a file containing a json-encoded dictionary
        of credentials.  If the file is successfully found and contains valid
        json, return the sub-dictionary based on `subset`.  If the file is not
        found, is not accessible, has improper encoding, or if the subset is 
        not present in the dictionary, return the given defaults.
    """
    try:
        with open(filename, 'r') as file:
            D = json.load(file)
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

creds = getCreds("creds.json", "mysql", defaults={'user': 'studen_j1', 
                                                  'password': 'studen_j1'})

In [ ]:
template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/'
cstring = template.format(creds['user'], creds['pass'])
cstring

## Module `sqlalchemy` Connection Establishment and First SQL

Given a connection string, the `sqlalchemy` module first creates a database "engine" that understands the SQL variation and the access library, as well as the other constituent parts of the connection string, which is really a database-specific URL.  In most instances, we will create a single connection to the database once we have created an engine, but, in general, the sqlalchemy abstraction allows the creation of *multiple* connections using the same engine.

In [ ]:
engine = sa.create_engine(cstring)
connection = engine.connect()

In its simplist form, a connection may be used as the channel over which we construct and send Python *strings*, whose contents consist of valid SQL.  In the case where we expect 0 rows as the result of an SQL, or where we are not interested in processing the result, we can just invoke with `execute` method of a connection with an argument of a Python string containing SQL.  In this particular example, we use SQL to set the default database currently being used by the connection.

In [ ]:
bookDB = "USE book"
connection.execute(bookDB)

We will discuss the `ResultProxy` more below, but for now, we issue a new SQL statement, one that retrieves all rows and all columns from the `indicators0` table, and we show the `fetchall()` method that, given the result of `execute()` ask for the ResultProxy to obtain all the rows satisfying the query.

In [ ]:
getind = "SELECT * FROM indicators0"
connection.execute(getind).fetchall()

Note that we get back a list of tuples, where each element of the list is a row of the result, and each tuple has the values for each of the columns in the table for that row.  If we wanted, we could assign this to a variable for further processing.

We are next going to show a variation where the default database is specified at the time we create our sqlalchemy engine.  Before we do, we want to show the important step of, given an open connection and instance of an engine, how do we gracefully close down the connection and release the resources being used by the engine.  It is acomplished with the two steps in the cell below, assuming the name of the connection is `connection` and the name of the engine is `engine`.  If we had chosen other variables for the names of those objects when we created and assigned them, we would use those variable names instead.

In [ ]:
connection.close()
del engine

If you like, you can check that your connection is really closed. For example, if you go back and run the previous cell, to select data from `indicators0`, you will now get an error.

## Connection Establishment and Basic Query II

Say that we know, at the time we establish an engine and a connection, that we want to access and use a particular database.  The connection string URL allows for this possiblity by specifing the name of the database *after* the name of the server machine.  The code in the cell below, assuming the credentials dictionary setup from the second cell of this notebook, uses Python string `format` to create a connection string that includes the database name in addition to the user and password strings from the `creds` dictionary. Note that the string stored in the variable `database` will be brought into `cstring` as the third blank spot in `template` (at the end). Note that you could also get a value of `database` from `creds.json` instead of typing it into this notebook.

In [ ]:
database = 'book'
template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
cstring = template.format(creds['user'], creds['pass'], database)
cstring

We then create the engine and connection as before.

In [ ]:
engine = sa.create_engine(cstring)
connection = engine.connect()

And now we can `execute()` an SQL statement without needing the `USE book` first.  In this case, we assign the return value of the `execute()` to the variable `resultproxy`.

In [ ]:
getind = "SELECT * FROM indicators0"
resultproxy = connection.execute(getind)

### Result Proxy Basics

The type of the return result from `execute()` is a `sqlalchemy.engine.result.ResultProxy`, an object class defined within `sqlalchemy`:

In [ ]:
type(resultproxy)

The ResultProxy is not yet the actuals rows resulting from the query, but we can find out, for instance, the column names of the result set by using the `keys()` method of a result proxy.

In [ ]:
resultproxy.keys()

Given a resultproxy, we can fetch rows of the result one at a time (using the `fetchone()` method), or as a batch (using the `fetchall()` method).  When we fetch one row, we get a special kind of tuple that has the values for all the columns from that row result.

In [ ]:
row = resultproxy.fetchone()
row

If we invoke `fetchall()`, we get a **list of tuples**, one tuple per row.  Note that, because we previously fetched one row, the `fetchall()` starts in the result from whereever we "left off", in this case at the row corresponding to `CHN`.

In [ ]:
rows = resultproxy.fetchall()
rows

Since the `rows` variable is a list, we can perform normal list-type operations.  One such operation (available on any Python list), is the `insert()` operation, which given a mutable object like `rows`, and a first argument that specifies the index before which we want the insert to occur, and a second argumnet that specifies the item to insert, results in the desired update of the list.

In [ ]:
rows.insert(0, row)
rows

Above, I stated that the rows of a query result were **special** tuples.  When we display such a row as an expression, or through a print, we get a normal-looking tuple.  But internally, these tuples each maintain the information about the column names of each of the tuple values.  If we invoke the `keys()` method on a **row**, we can get the column names:

In [ ]:
row.keys()

These special tuples also have an `items()` method, that allows us to iterate and get both the column name and the column value for each field in the tuple.

In [ ]:
for column, value in row.items():
    template = "column name: {:>10}, value: {}"  # Note the format syntax to right justify in a field width of 10
    s = template.format(column, value)
    print(s)

### Building a DataFrame and Writing a Function

We often would prefer a `pandas` DataFrame, instead of a list of tuples, for working with the result of an SQL query.  From the `pandas` point of view, a list of tuples and a list of lists provide the same data representation of a list of rows containing the data.  Just like we did with a list of lists and then a list of column names, we can create a `pandas` DataFrame as follows.

In [ ]:
df = pd.DataFrame(rows, columns=resultproxy.keys())
df

It is bad form to perform this set of operations in global cells.  By the principle of functional abstraction, we want an **abstraction** that retrieves the rows and columns of the Teams table.  The function should be parameterized by an `sqlalchemy` connection, since we should not reference a global variable in the body of our function.  Other than that, the sequence of statements in the body of the function look much like our global cell experimentation:

In [ ]:
def get_indicators(db_connection):
    """ Retrieve the indicators0 table, given a live connection 
    to the database, as given in the `db_connection` parameter.
        
    Assumes that book is the current default database
    set up on the connection.
    
    Returns a pandas dataframe with all rows and all columns 
    from the table.
    """
    
    # Set up the SQL query
    getind = "SELECT * FROM indicators0"
    
    # Execute the query and fetch all rows of the result
    resultproxy = db_connection.execute(getind)
    resultset = resultproxy.fetchall()
    
    # Construct the pandas dataframe from the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

Now, whenever we want to get the `indicators0` table in a dataframe, we call our new function:

In [ ]:
df = get_indicators(connection)
df

The `pandas` module actually understands `sqlalchemy` connections, and provides a module function that takes a Python string with an SQL query as its contents and builds a DataFrame from the result, as shown below.  Until further notice, however, since our goal is to understand the principles of working with a relational database from a programming language, the exclusive use of this shortcut is disallowed in homeworks and projects over the duration of this course unit. That is, you should still expect to have to solve problems as laid out above.

In [ ]:
df = pd.read_sql_query('SELECT * FROM indicators0', connection)
df

We clean up our connection and engine, in preparation for another setup example.

In [ ]:
connection.close()
del engine

## Connection Establishment III

We can place the steps for setting up a connection and engine into its own function, so that we can reuse the code and can refrain from doing everything at the global cell level.  The parameters needed include the strings needed for the user, the password, and the default database.

In [ ]:
def db_setup(user, password, database):
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(user, password, database)
    e = sa.create_engine(cstring)
    c = e.connect()
    return e, c

Since we need to make sure any previous engine and connection are cleaned up before we create new objects, the following cell attempts to close the connection and delete the engine.  This is placed in a Python `try`-`except` block, which will allow the code to succeed, even if there is not an open connections and an existant engine object.  So this cell is safe to "repeat"-execute whenever we want to establish or change the default database.

In this case, we re-estblish a connection to the same database.

In [ ]:
try:
    connection.close()
    del engine
except:
    pass
engine, connection = db_setup(creds['user'], creds['pass'], 'book')

If we have an `engine` object, and it is associated with a default database, we can programmatically find out the set of tables present in the database.  This can be more convenient than using an external program like MySQL Workbench.  This is done with the `table_names()` method.

In [ ]:
engine.table_names()

We can also query whether or not a particular table is present in the database with the `has_table()` method.

In [ ]:
engine.has_table('indicators0')

In [ ]:
engine.has_table('Tom')

If are unsure of the column fields of a particular table, we can request a projection that includes all columns but, by virtue of a `LIMIT` requests no rows.  Even if we get no rows, we still get a result proxy, and the `keys()` method still works.  So below, I define a quick function that returns a list of column names for a given table name on a given database connection:

In [ ]:
def getIndColumns(conn, table):
    sql_template = "SELECT * FROM {} LIMIT 0"
    query = sql_template.format(table)
    resultproxy = conn.execute(query)
    return resultproxy.keys()

In [ ]:
getIndColumns(connection, "indicators0")

## Building SQL Queries that use values of Python variables

When integrating programmatic problem solving with database access and queries, we need to incorporate values associated with Python variables in our programs with the SQL statements we are constructing before submitting them to the SQL server for execution.  In this section, we will explore a two mechanisms for accomplishing this:
- Using a **variable binding** facility of the `sqlalchemy` moduleon the client side and generally supported by the various flavors of SQL servers.
- Using basic facilities of Python string processing to incorporate the variable values into a composed string and then executing the string as we have above.

### Binding Variables

In the cell below, we use a building block of the `text()` method of `sqlalchemy`'s `sql` class.  This allows us to use a Python string as an argument and construct an object capable of understanding SQL queries that include notation for **variable binding**, which will allow us to dynamically "substitute" values of variables at defined places within the query.  In the example below, the SQL syntax is the same as we have learned except for the `:x` and `:y` in the query string.  These define the locations for substitution and, in addition, give symbolic names (`x` and `y`) that enable us to distinguish when we have more than one substitution location within the query.

In [ ]:
stmt = sa.sql.text("SELECT * FROM indicators0 WHERE life BETWEEN :x AND :y")

Given the SQL text statement object above, we can make a call to `execute()` and use **named parameters**, where the parameter argument name matches the symbolic name given in the query string, and the value is whatever value we wish from our program, from constant literals, like shown below, to program variables, to arbitrary Python expressions yielding the desired value.  In the example below we are doing a combination of executing and **binding** of value `70` to `x` and `80` to `y` to get all countries whose life expectancy is between 70 and 80 (inclusive).

In [ ]:
connection.execute(stmt, x=70, y=80).fetchall()

Since the variable `stmt` refers to an **unbound** query, we can reuse it later, with different binding to `x` and `y`, but without reconstructing `stmt`.  This promotes reusability of the so-called **paramaterized query** in multiple places in the program, and also helps readabilty, since we can see what parts of the query stay the same, and what parts may vary with arguments.

Given an unbound query, we can explicitly make the step of binding values to the query parameters as shown in the cell below.  Note that we assign to a new variable, so that we can still have the unbound version of the query.  Also note, from the print, that the binding operation does not actually perform the subsitution within the query string.  Given a bound query, it is at the **server** that binding takes place.  This actually allows for better performing sequences of SQL queries, as the server can optimize and cache "pre-compiled" portions of the server execution based on the unbound portion.

Our next example shows that how to bind string variables into a statement. This query finds all countries whose code starts with A,B,C, or D.

In [ ]:
stmt2 = sa.sql.text("SELECT * FROM indicators0 WHERE code BETWEEN :x AND :y")
stmtbound = stmt2.bindparams(x="A", y="Dz")
print(stmtbound)

If we have a bound version of a parameterized SQL query, we can invoke `execute()`, and omit the named parameters.  We can also, at that point, take advantage of the `pandas` `read_sql_query` function that can perform the execution and build a DataFrame on our behalf.

In [ ]:
pd.read_sql_query(stmtbound, connection)

Variable binding in our SQL queries can also give us a *clean* way to build functional abstractions that nicely generalize operations we wish to perform against a database.  Instead of using global variables and global cells to perform the above, we should always write a function.  In this case, since we want the function to take an arbitrary **"from"** and **"to"** to specify the range of names, we add parameters for each of these, and use the  techniques from above to compose a function that gets all the columns from the `indicators` table (not `indicators0`) for country codes in the range `namefrom` to `nameto`.

In [ ]:
def getIndicatorsInNameRange(connection, namefrom, nameto):
    """ Retrieve a subset of the indicators table, 
    given a live connection to the database.
    Assumes that book is the current default database
    set up on the connection.
    Returns a pandas dataframe with all columns from 
    the table, and whose codes are
    in the inclusive range from `namefrom` to `nameto`
    """
    getIndicators = sa.sql.text("SELECT * FROM indicators WHERE code BETWEEN :x AND :y")
    boundGetInd = getIndicators.bindparams(x=namefrom, y=nameto)
    resultproxy = connection.execute(boundGetInd)
    resultset = resultproxy.fetchall()
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

Invocation of our new function can then be performed any time we want this functionality and can use different argments to specify different name ranges.

In [ ]:
getIndicatorsInNameRange(connection, "H", "Kz")

### Python string `format()` composition

The same resultant effect can be obtained by using Python string manipulation to actually build the complete query string, with programmatic techniques used to take the values from our Python program and substitute them into the string to form the query.  This complete query string can then be sent and executed at the server.

In the following cell, we show how we can use the Python string `format()` method, which substitutes values from the arguments to format into a template string.  The location of the substitution/conversion is specified using curly braces `{}` in the template string.  There is an intentional error in this code.  Execute the definition cell and then the invocation cell and see if you can figure out the problem and correct it. Hint: look at the very bottom of the error message and notice what's wrong with that query.

In [ ]:
def getIndicatorsInNameRange2(connection, namefrom, nameto):
    """ Retrieve a subset of the indicators table, 
    given a live connection to the database.
    Assumes that book is the current default database
    set up on the connection.
    Returns a pandas dataframe with all columns from 
    the table, and whose codes are
    in the inclusive range from `namefrom` to `nameto`
    """
    getIndicators = "SELECT * FROM indicators WHERE code BETWEEN {} AND {}"
    formatGetInd = getIndicators.format(namefrom, nameto)
    resultproxy = connection.execute(formatGetInd)
    resultset = resultproxy.fetchall()
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df


In [ ]:
getIndicatorsInNameRange2(connection, "H", "Kz")

Now we take a similar example, where we want to specify a range of integers to use to get countries with `life` in a specified range. The first function uses the SQL bind variables approach, and it looks much like the function above:

In [ ]:
def getLifeInRange(connection, lifefrom, lifeto):
    """ Retrieve a subset of the indicators table, given 
    a live connection to the database.
    Assumes that book is the current default database
    set up on the connection.
    Returns a pandas dataframe with all columns from the 
    table, and whose life expectancies are
    in the inclusive range from `lifefrom` to `lifeto`
    """
    getCountries = sa.sql.text("SELECT * FROM indicators WHERE life BETWEEN :x AND :y")
    boundgetCountries = getCountries.bindparams(x=lifefrom, y=lifeto)
    resultproxy = connection.execute(boundgetCountries)
    resultset = resultproxy.fetchall()
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [ ]:
getLifeInRange(connection, 70, 80)

In the following cell, we repeat the exercise from above -- namely using Python string manipulation to create a string query without bind variables and execute it.  Note that this version is just like the uncorrected `getIndicatorsInNameRange2()` function above.  What happens if you execute this without change?

In [ ]:
def getIndicatorsInNameRange2(connection, lifefrom, lifeto):
    """ Retrieve a subset of the indicators table, given 
    a live connection to the database.
    Assumes that book is the current default database
    set up on the connection.
    Returns a pandas dataframe with all columns from the 
    table, and whose life expectancies are
    in the inclusive range from `lifefrom` to `lifeto`
    """
    getCountries = "SELECT * FROM indicators WHERE life BETWEEN {} AND {}"
    formatGetCountries = getCountries.format(lifefrom, lifeto)
    resultproxy = connection.execute(formatGetCountries)
    resultset = resultproxy.fetchall()
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df


In [ ]:
getIndicatorsInNameRange2(connection, 70, 80)

Now make the same change you made to correct `getIndicatorsInNameRange2()` in `getLifeInRange2()` above and reexecute.  What results do you get?

The point of this exercise is to emphasize that **data types matter.** . If we have string data types in an SQL query, we have to make sure a variable substituted into a query has the single quotes surrounding it, if we are doing the string format method.  If we are using bind variables, the underlying library/software (both on the client side and on the server side) understands the data type of the variables being bound, and accommodates appropriately.  This is one factor in favor of using bind variables.

The last example shows a combination of string processing along with still using bind variables.  Say that you want a functional abstraction that paramerizes the life expectancy range to select from the `indicators` table, but sometimes you want all columns projected, and other times you want a subset of the columns.  To have a single function that enables both, we can add a (string) parameter that abstracts the desired columns.  In the function definition, we can have a default of `*` to select all columns.  Then, if we do not include an argument for `cols`, we get the same behavior as before.  But if we specify a `cols` argument, the function first uses string manipulation (the `.format()` method) to compose the query string, including the variable bindings.  We then proceed the same way we did in the previous bind variables case.

In [ ]:
def getLifeInRange3(connection, lifefrom, lifeto, cols='*'):
    """ Retrieve a subset of the indicators table, given a live connection to the database.
    Assumes that book is the current default database
    set up on the connection.
    Returns a pandas dataframe with all columns from the table, 
    and whose life expectancies are
    in the inclusive range from `lifefrom` to `lifeto`
    """
    
    # Set up the SQL Query
    getLife0 = "SELECT {} FROM indicators WHERE life BETWEEN :x AND :y".format(cols)
    getLife = sa.sql.text(getLife0)
    boundGetLife = getLife.bindparams(x=lifefrom, y=lifeto)
    
    # Execute and fetch all rows of the result
    resultproxy = connection.execute(boundGetLife)
    resultset = resultproxy.fetchall()
    
    # Build a pandas dataframe using the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [ ]:
getLifeInRange3(connection, 60, 65, "code, year, life")

The default value for `cols` in the function above is `*` meaning if you leave off the variable `cols` when calling `getLifeInRange3` then it'll select all columns.

In [ ]:
getLifeInRange3(connection, 60, 70)